# Lesson 6 : EPICS area detector

Start the instrument package as our routine initialization.

In [1]:
from instrument.collection import *

I Fri-17:48:42 - ############################################################ startup
I Fri-17:48:42 - logging started
I Fri-17:48:42 - logging level = 10
I Fri-17:48:42 - /home/mintadmin/Documents/projects/use_bluesky/lessons/instrument/collection.py
I Fri-17:48:42 - /home/mintadmin/Documents/projects/use_bluesky/lessons/instrument/mpl/notebook.py


Activating auto-logging. Current session state plus future input saved.
Filename       : /home/mintadmin/Documents/projects/use_bluesky/lessons/.logs/ipython_console.log
Mode           : rotate
Output logging : True
Raw input log  : False
Timestamping   : True
State          : active


I Fri-17:48:42 - bluesky framework
I Fri-17:48:42 - /home/mintadmin/Documents/projects/use_bluesky/lessons/instrument/framework/check_python.py
I Fri-17:48:42 - /home/mintadmin/Documents/projects/use_bluesky/lessons/instrument/framework/check_bluesky.py
I Fri-17:48:43 - /home/mintadmin/Documents/projects/use_bluesky/lessons/instrument/framework/initialize.py
I Fri-17:48:44 - /home/mintadmin/Documents/projects/use_bluesky/lessons/instrument/framework/metadata.py
I Fri-17:48:44 - /home/mintadmin/Documents/projects/use_bluesky/lessons/instrument/framework/callbacks.py
I Fri-17:48:44 - writing to SPEC file: /home/mintadmin/Documents/projects/use_bluesky/lessons/20200508-174844.dat
I Fri-17:48:44 -    >>>>   Using default SPEC file name   <<<<
I Fri-17:48:44 -    file will be created when bluesky ends its next scan
I Fri-17:48:44 -    to change SPEC file, use command:   newSpecFile('title')


The EPICS area detector support in ophyd has many features, some of them specific to the detector make and model.  Let's start with just a few features using the ADSimDetector.  We'll need to import support from ophyd by parts as they are needed.

As we add features from the area detector plugins or other capabilities, the configuration complexity will increase.  So, it is good to start with a simple case where we can control the camera and generate images.

In [2]:
from ophyd import Component
from ophyd.areadetector import ImagePlugin
from ophyd.areadetector import SimDetector
from ophyd.areadetector import SingleTrigger

In [3]:
_ad_prefix = "adsky:"

In [4]:
class MySingleTriggerSimDetector(SingleTrigger, SimDetector): 
       
    image = Component(ImagePlugin, suffix="image1:")

In [5]:
adsimdet = MySingleTriggerSimDetector(_ad_prefix, name='adsimdet')
adsimdet.wait_for_connection()

In [6]:
adsimdet.stage_sigs["cam.num_images"] = 1
adsimdet.stage_sigs["cam.acquire_time"] = 0.1
adsimdet.stage_sigs["cam.acquire_period"] = 0.25

adsimdet.stage_sigs

OrderedDict([('cam.acquire', 0),
             ('cam.image_mode', 1),
             ('cam.num_images', 1),
             ('cam.acquire_time', 0.1),
             ('cam.acquire_period', 0.25)])

In [8]:
RE(bp.count([adsimdet]))

Run aborted
Traceback (most recent call last):
  File "/home/mintadmin/Apps/anaconda/envs/bluesky_2020_5/lib/python3.7/site-packages/bluesky/run_engine.py", line 1355, in _run
    msg = self._plan_stack[-1].send(resp)
  File "/home/mintadmin/Apps/anaconda/envs/bluesky_2020_5/lib/python3.7/site-packages/bluesky/preprocessors.py", line 1307, in __call__
    return (yield from plan)
  File "/home/mintadmin/Apps/anaconda/envs/bluesky_2020_5/lib/python3.7/site-packages/bluesky/preprocessors.py", line 1160, in baseline_wrapper
    return (yield from plan)
  File "/home/mintadmin/Apps/anaconda/envs/bluesky_2020_5/lib/python3.7/site-packages/bluesky/preprocessors.py", line 803, in monitor_during_wrapper
    return (yield from plan2)
  File "/home/mintadmin/Apps/anaconda/envs/bluesky_2020_5/lib/python3.7/site-packages/bluesky/preprocessors.py", line 170, in plan_mutator
    raise ex
  File "/home/mintadmin/Apps/anaconda/envs/bluesky_2020_5/lib/python3.7/site-packages/bluesky/preprocessors.py", 

TimeoutError: Attempted to set EpicsSignalWithRBV(read_pv='adsky:cam1:Acquire_RBV', name='adsimdet_cam_acquire', parent='adsimdet_cam', value=1, timestamp=1588978363.272837, auto_monitor=False, string=False, write_pv='adsky:cam1:Acquire', limits=False, put_complete=False) to value 0 and timed out after 10 seconds. Current value is 1.

Get the detector image and display it.  In the initial instrument setup, MatPlotLib was initialized for our display.  We need a copy of the plotting object that was created, `plt` to show the detector image.

In [ ]:
from instrument.mpl import plt

# The image comes from EPICS AD as a 1-D array, row by row.
# We need to make it into a 2-D array, so first we need to
# get the number of rows and columns from the image plugin.

array_size = adsimdet.image.array_size.get()
shape = (array_size.height, array_size.width)

# Now, get the image and make it a 2-D array.
im = adsimdet.image.array_data.get().reshape(shape)

# Tell MatPlotLib to show the image.
plt.imshow(im)

It's useful to make these steps into a Python function.

In [ ]:
def show_ad_image(det):
    array_size = det.image.array_size.get()
    shape = (array_size.height, array_size.width)
    im = det.image.array_data.get().reshape(shape)
    plt.imshow(im)

In [ ]:
plt.close()   # stop interaction with previous MPL window
show_ad_image(adsimdet)

----

Save image(s) to HDF5 file.

In [ ]:
from ophyd.areadetector import ADComponent
from ophyd.areadetector import EpicsSignalWithRBV
from ophyd.areadetector import HDF5Plugin
from ophyd.areadetector.filestore_mixins import FileStoreHDF5IterativeWrite

class MyHDF5Plugin(HDF5Plugin, FileStoreHDF5IterativeWrite):
    create_directory_depth = Component(EpicsSignalWithRBV, suffix="CreateDirectory")
    array_callbacks = Component(EpicsSignalWithRBV, suffix="ArrayCallbacks")

    pool_max_buffers = None
    
    def get_frames_per_point(self):
        return self.num_capture.get()

    def stage(self):
        super().stage()
        res_kwargs = {'frame_per_point': self.get_frames_per_point()}
        # res_kwargs = {'frame_per_point': self.num_capture.get()}
        self._generate_resource(res_kwargs)

In [ ]:
DATABROKER_ROOT_PATH = "/tmp/"

# note: AD path MUST, must, MUST have trailing "/"!!!
#  ...and... start with the same path defined in root (above)

# path as seen by detector IOC
WRITE_HDF5_FILE_PATH = "/tmp/simdet/%Y/%m/%d/"
#!!! NOTE !!! This filesystem is on the IOC (might be in a docker container)!

# path as seen by databroker
READ_HDF5_FILE_PATH = "/tmp/adsky/simdet/%Y/%m/%d/"

In [ ]:
class MySingleTriggerSimDetector(SingleTrigger, SimDetector): 
       
    image = Component(ImagePlugin, suffix="image1:")
    hdf1 = ADComponent(
        MyHDF5Plugin,
        suffix='HDF1:', 
        root=DATABROKER_ROOT_PATH,
        write_path_template = WRITE_HDF5_FILE_PATH,
        read_path_template = READ_HDF5_FILE_PATH,
    )

In [ ]:
adsimdet = MySingleTriggerSimDetector(_ad_prefix, name='adsimdet')
adsimdet.stage_sigs["cam.num_images"] = 1
adsimdet.stage_sigs["cam.acquire_time"] = 0.1
adsimdet.stage_sigs["cam.acquire_period"] = 0.25
adsimdet.hdf1.stage_sigs["num_capture"] = 1

adsimdet.read_attrs.append("hdf1")
if adsimdet.hdf1.create_directory_depth.get() == 0:
    # probably not set, so let's set it now to some default
    adsimdet.hdf1.create_directory_depth.put(-5)

NOTE: EPICS AreaDetector file-saving plugins (such as 
the HDF plugin) must be primed before they can be 
used.  Priming must be done in these situations:

* the IOC has just started
* the image size has changed

If you do not prime the detector and its plugin chain, *ophyd* will report an *UnprimedPlugin* error like this if you try to trigger (acquire data from) the detector:

```
UnprimedPlugin: The plugin hdf1 on the area detector with name DETECTOR_NAME has not been primed.
```

To prime the HDF plugin, call its `warmup()` method.

In [ ]:
enabled = adsimdet.hdf1.enable.get()
adsimdet.hdf1.warmup()
adsimdet.hdf1.enable.put(enabled)

In [ ]:
adsimdet.hdf1.stage_sigs

In [ ]:
print(f"frames/point: {adsimdet.hdf1.get_frames_per_point()}")
RE(bp.count([adsimdet]))

----

Now, use databroker to retrieve that data and show the image.

In [ ]:
h = db[-1]

In [ ]:
plt.close()   # stop interaction with previous MPL window
plt.imshow(h.xarray().adsimdet_image[0][0])